# **Identificando Equipes de Jogadores de Futebol usando Distâncias**

*Grupo: John Wesley Santos Alencar, Fabio Cardoso, Luiz Felipe Menezes Barbosa, Rogério Fernandes*



# Importação das bibliotecas utilizadas

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from scipy.spatial import distance
import requests
from PIL import Image
from io import BytesIO

# Importando o dataset do github e tratando um pouco os dados

In [ ]:
# Pegando o dataset do github
url = "https://raw.githubusercontent.com/devjaynemorais/modelos_descritivos_curso/main/Aula%2001/dados/players_match.csv"
df = pd.read_csv(url)

In [ ]:
df.head(100)

,Unnamed: 0,Player_Position_X,Player_Position_Y,Center_Side_A_X,Center_Side_A_Y,Center_Side_B_X,Center_Side_B_Y
0,0,8.107316,13.362118,25.0,50.0,75,50
1,1,48.687401,80.312717,25.0,50.0,75,50
2,2,31.740322,64.865129,25.0,NaN,75,50
3,3,13.652919,81.462626,25.0,50.0,75,50
4,4,19.134233,28.985857,25.0,50.0,75,50
5,5,16.793206,97.194354,NaN,50.0,75,50
6,6,18.269684,56.269861,25.0,50.0,75,50
7,7,14.532925,18.316240,25.0,50.0,75,50
8,8,16.813938,78.656237,25.0,50.0,75,50
9,9,37.347460,2.736767,25.0,50.0,75,50


# Realizando Drop, Fillna e Rename

In [ ]:
#alterando o nome da primeira coluna para player_name
df = df.rename(columns={'Unnamed: 0': 'player_name'})

# Excluindo linhas que estão com as coordenadas de player x e y vazias.
#Ao realizar o drop e realizar um count() na coluna player_name o dataset fica exatamente com 22 linhas
df.drop([24, 25, 26, 11, 12], inplace=True)

# Preenchendo os valores ausentes nas colunas Center_Side_A_X e Y com a média da coluna
#como todas as linhas possuem os mesmos valores, a média continua 25.0 para X e 50.0 para Y
df['Center_Side_A_X'].fillna(df['Center_Side_A_X'].mean(), inplace=True)
df['Center_Side_A_Y'].fillna(df['Center_Side_A_Y'].mean(), inplace=True)

In [ ]:
df.head(100)

,player_name,Player_Position_X,Player_Position_Y,Center_Side_A_X,Center_Side_A_Y,Center_Side_B_X,Center_Side_B_Y
0,0,8.107316,13.362118,25.0,50.0,75,50
1,1,48.687401,80.312717,25.0,50.0,75,50
2,2,31.740322,64.865129,25.0,50.0,75,50
3,3,13.652919,81.462626,25.0,50.0,75,50
4,4,19.134233,28.985857,25.0,50.0,75,50
5,5,16.793206,97.194354,25.0,50.0,75,50
6,6,18.269684,56.269861,25.0,50.0,75,50
7,7,14.532925,18.316240,25.0,50.0,75,50
8,8,16.813938,78.656237,25.0,50.0,75,50
9,9,37.347460,2.736767,25.0,50.0,75,50


In [ ]:
df['player_name'].count()

22

# Definindo os jogadores para time A e B usando a distância euclidiana

In [ ]:
# Coordenadas dos centros
center_a = (25, 50)
center_b = (75, 50)

# Calculando as distâncias usando a métrica de distância euclidiana
df['Distance_A'] = df.apply(lambda row: distance.euclidean((row['Player_Position_X'], row['Player_Position_Y']), center_a), axis=1)
df['Distance_B'] = df.apply(lambda row: distance.euclidean((row['Player_Position_X'], row['Player_Position_Y']), center_b), axis=1)

#Determinando a equipe com base nas distâncias
#Basicamente todos os jogadores são do time 'A', porém, utilizo loc
#para atualizar a coluna 'Team' com 'B, para os jogadores em que a distancia
# até o centro B forem menor que a distância até o centro A.
df['Team'] = 'A'
df.loc[df['Distance_B'] < df['Distance_A'], 'Team'] = 'B'

# Baixando a imagem de fundo

In [ ]:
# Baixando a imagem do campo de futebol
image_url = "https://i.ibb.co/h704wMC/TAPETE-CAMPO-RETANGULAR.jpg"
response = requests.get(image_url)
img = Image.open(BytesIO(response.content))

# Realizando o plot do gráfico com plotly GO

In [ ]:
#criando o gráfico com o go
fig = go.Figure()

#Adicionando bolinhas dos jogadores do Time A
fig.add_trace(go.Scatter(
    x=df[df['Team'] == 'A']['Player_Position_X'],
    y=df[df['Team'] == 'A']['Player_Position_Y'],
    mode='markers+text',
    text=df[df['Team'] == 'A']['player_name'],
    marker=dict(color='orange', size=20),
    name='Time A'
))

#Adicionando bolinhas dos jogadores do Time B
fig.add_trace(go.Scatter(
    x=df[df['Team'] == 'B']['Player_Position_X'],
    y=df[df['Team'] == 'B']['Player_Position_Y'],
    mode='markers+text',
    text=df[df['Team'] == 'B']['player_name'],
    marker=dict(color='lightskyblue', size=20),
    name='Time B'
))

#Definindo a imagem do campo de futebol como plano de fundo
fig.add_layout_image(
    source=img,
    x=0,
    y=100,
    xref='x',
    yref='y',
    sizex=100,
    sizey=100,
    sizing='stretch',
    opacity=0.3,
    layer='below'
)

#Adicionando marcação dos centros A e B
fig.add_trace(go.Scatter(x=[center_a[0]], y=[center_a[1]], mode='markers', marker=dict(symbol="x", size=10, color='red'), name='Centro A'))
fig.add_trace(go.Scatter(x=[center_b[0]], y=[center_b[1]], mode='markers', marker=dict(symbol="x", size=10, color='blue'), name='Centro B'))

#Atualizando o layout para ajustar o tamanho da figura e a legenda
fig.update_layout(
    title='Posição dos Jogadores de Futebol no Campo',
    xaxis=dict(title='Posição X'),
    yaxis=dict(title='Posição Y'),
    width=1200,
    height=720,
    legend=dict(title='Legenda', orientation='h')
)

# Exibindo o gráfico interativo
fig.show()
